<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/ipynb/Lesson%2025/25_4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Первое знакомство с AutoML](https://colab.research.google.com/drive/1bCWyzlp1-tcvt7TE60m4hFnRd5AWscWY)
2. [Гиперпараметры и оптимизация моделей](https://colab.research.google.com/drive/1CN69NftfVXUliyv11FGbM7qOYbO0XON5)
3. [AutoML в Keras](https://colab.research.google.com/drive/1V7mfY8da0S-FbWxhQbchJM38JSJBmtoZ)
4. Домашняя работа

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [2]:
!wget https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
!unzip -qo "Sarcasm_Headlines_Dataset_v2.json.zip" -d ./dataset

--2025-01-21 16:35:24--  https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1789636 (1.7M) [application/x-zip-compressed]
Saving to: ‘Sarcasm_Headlines_Dataset_v2.json.zip.2’

Sarcasm_Headlines_D 100%[===================>]   1.71M  5.18MB/s    in 0.3s    

2025-01-21 16:35:25 (5.18 MB/s) - ‘Sarcasm_Headlines_Dataset_v2.json.zip.2’ saved [1789636/1789636]



In [3]:
!pip install tensorflow==2.15.1

In [4]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

In [5]:
import tensorflow as tf
tf.__version__

'2.15.1'

In [6]:
import keras_nlp as nlp
nlp.__version__

'0.5.1'

In [7]:
# Библиотека матричного вычисления
import numpy as np
# Библиотека для работы с данными
import pandas as pd
# Библиотека для работы с регулярными выражениями
import re
# Библиотека для работы с фреймворком TensorFlow
import tensorflow as tf
# Библиотека AutoML autokeras
import autokeras as ak
# Библиотеки для построения графиков и их стилизации
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

# Необходимые метрики для построения Матрицы ошибок и отчета о классификации
from sklearn.metrics import classification_report, confusion_matrix


import tensorflow as tf
import keras_nlp as nlp
import numpy as np
import pandas as pd
import re
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.src.utils import to_categorical

In [8]:
address = "./dataset/Sarcasm_Headlines_Dataset_v2.json"
json_df = pd.read_json(address, lines = True ) # библиотека pandas умеет работать с json данными
df_sarcasm = pd.DataFrame(json_df) # создаем датафрейм

df_sarcasm.head() # выводим первые 5 записей датафрейма

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [9]:
# Ссылки на сами статьи нам не нужны
df_sarcasm = df_sarcasm.drop("article_link", axis = 1)

In [10]:
print('Найдено дубликатов: ', df_sarcasm.duplicated().sum())

# Удаляем дубликаты
df_sarcasm.drop_duplicates(subset=['headline'], inplace = True)

print('Осталось дубликатов после очистки: ', df_sarcasm.duplicated().sum())

Найдено дубликатов:  116
Осталось дубликатов после очистки:  0


In [11]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join(text.split())
    return text


df_sarcasm["headline"] = df_sarcasm["headline"].apply(preprocess_text)
df_sarcasm.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep totally nails why congress is falling ...
2,0,eat your veggies deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word stream...


In [12]:
# prompt: remover rare words from `df_sarcasm["headline"]`

from collections import Counter

# Count word frequencies
word_counts = Counter()
for headline in df_sarcasm["headline"]:
    words = headline.split()
    word_counts.update(words)

# Define a threshold for rare words (e.g., words appearing less than 5 times)
threshold = 5

# Remove rare words from headlines
def remove_rare_words(headline):
    words = headline.split()
    filtered_words = [word for word in words if word_counts[word] >= threshold]
    return " ".join(filtered_words)

df_sarcasm["headline"] = df_sarcasm["headline"].apply(remove_rare_words)
df_sarcasm.head()

,is_sarcastic,headline
0,1,scientists unveil doomsday clock of hair loss
1,0,rep totally nails why congress is falling shor...
2,0,eat your different recipes
3,1,weather prevents liar from getting to work
4,1,mother comes pretty close to using word stream...


In [13]:
X = (np.array(df_sarcasm.headline),)
y = (np.array(df_sarcasm.is_sarcastic),)

target = df_sarcasm.is_sarcastic
from keras.src.utils import to_categorical
y = to_categorical(target)


X_train, X_tmp, y_train, y_tmp = train_test_split(
    np.array(df_sarcasm.headline),
    np.array(df_sarcasm.is_sarcastic),
    test_size=0.1,
)

X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5)

In [14]:
print('Форма входных данных: ', X_train.shape)
print('Форма выходных меток: ', y_train.shape)
print('Пример заголовка: ', X_train[0])

Форма входных данных:  (25652,)
Форма выходных меток:  (25652,)
Пример заголовка:  weekly roundup of ebay vintage home finds


In [62]:
def check_class_balance(y):
    unique, counts = np.unique(y, return_counts=True)
    balance_dict = dict(zip(unique, counts))
    print("Class distribution:")
    for label, count in balance_dict.items():
        print(f"Class {label}: {count} samples ({count/len(y)*100:.2f}%)")
    return balance_dict


print("\nTraining set class balance:")
train_balance = check_class_balance(y_train)

print("\nValidation set class balance:")
val_balance = check_class_balance(y_val)

print("\nTest set class balance:")
test_balance = check_class_balance(y_test)


Training set class balance:
Class distribution:
Class 0: 13509 samples (52.45%)
Class 1: 12248 samples (47.55%)

Validation set class balance:
Class distribution:
Class 0: 743 samples (51.92%)
Class 1: 688 samples (48.08%)

Test set class balance:
Class distribution:
Class 0: 733 samples (51.22%)
Class 1: 698 samples (48.78%)


In [40]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF vectorizer to convert text to numerical features
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform it
X_train_vec = vectorizer.fit_transform(X_train)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the numerical features
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Print the resampled training set class balance
print("\nResampled Training set class balance:")
check_class_balance(y_train_resampled)

# Update X_train and y_train with the resampled data
# X_train = X_train_resampled # Keep the data in the vectorized format
# y_train = y_train_resampled


Resampled Training set class balance:
Class distribution:
Class 0: 11928 samples (50.00%)
Class 1: 11928 samples (50.00%)


{0: 11928, 1: 11928}

In [41]:
print('Форма входных данных: ', X_train_resampled.shape)
print('Форма выходных меток: ', y_train_resampled.shape)
print('Пример заголовка: ', X_train_resampled[0])

Форма входных данных:  (23856, 25782)
Форма выходных меток:  (23856,)
Пример заголовка:    (0, 13290)	0.3234121009395452
  (0, 3496)	0.37522726559476055
  (0, 1338)	0.41081896911282395
  (0, 25462)	0.2956550240398538
  (0, 18597)	0.3666417293603352
  (0, 8747)	0.16335897771166197
  (0, 23012)	0.3113140502882134
  (0, 19876)	0.4562616086520702
  (0, 15912)	0.17950372350805124


In [63]:
# Example usage: Print the first 5 elements to verify
print(X_train[:5])

['evangelical voters dont care that trumps not religious'
 'struggling don rickles has nothing but nice things to say about audience'
 'local los angeles awards show slated to open for grammys'
 'argentinian tv station trolls donald trump'
 'professor pressured to sleep with student for good course evaluation']


In [46]:
# prompt: X_train_resampled back to string

# Convert the sparse matrix X_train_resampled back to a list of strings
X_train_temp = ' '.join(vectorizer.inverse_transform(X_train_resampled))
print(X_train_temp[:3])

TypeError: sequence item 0: expected str instance, numpy.ndarray found

In [48]:
# Для экономии ОЗУ удаляем уже ненужные данные, после чего python запускает сборщик "мусора" для очистки памяти
del df_sarcasm, X_tmp, y_tmp

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer to the training data and transform it
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)
X_test_bow = vectorizer.transform(X_test)

# Print the bag-of-words representation (optional)
print(X_train_bow.toarray())

In [15]:
# Создание ансамбля моделей (3 наиболее подходящие к данным моделей)
clf = ak.TextClassifier(overwrite=True, max_trials=2, objective='val_accuracy')

# Обучаем 2 модели на 4-х эпохах, размер пакета подбирается автоматически
result_training = clf.fit(X_train, y_train, epochs=4, validation_data=(X_val, y_val))


Trial 2 Complete [00h 02m 15s]
val_accuracy: 0.8470175266265869

Best val_accuracy So Far: 0.8512280583381653
Total elapsed time: 00h 14m 16s
Epoch 1/4
802/802 [==============================] - 184s 227ms/step - loss: 0.4330 - accuracy: 0.7880 - val_loss: 0.3311 - val_accuracy: 0.8498
Epoch 2/4
802/802 [==============================] - 176s 220ms/step - loss: 0.2580 - accuracy: 0.8941 - val_loss: 0.3497 - val_accuracy: 0.8463
Epoch 3/4
802/802 [==============================] - 176s 220ms/step - loss: 0.1605 - accuracy: 0.9394 - val_loss: 0.4560 - val_accuracy: 0.8302
Epoch 4/4
802/802 [==============================] - 178s 222ms/step - loss: 0.1044 - accuracy: 0.9616 - val_loss: 0.5711 - val_accuracy: 0.8316


In [17]:
# Save the model
model = clf.export_model()
model.save("sarcasm_detection_model")

# Save hyperparameters
best_hyperparameters = clf.tuner.get_best_hyperparameters()[0]
with open("best_hyperparameters.txt", "w") as f:
  f.write(str(best_hyperparameters.values))

In [18]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print("Точность на тестовых данных:", acc)
print("Потери на тестовых данных:", loss)

Точность на тестовых данных: 0.8330996036529541
Потери на тестовых данных: 0.5903888940811157
